In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [13]:
from utils.cluster.similarity_metrics import intra_pairs, intra_pairs_similarity

lhs = [0, 0, 1, 1, 1, 2, 3, 3]
rhs = [0, 0, 1, 1, 1, 2, 2, 2]

print(f'intra-pairs_old(lhs) -> {" ".join([f"<{l}, {r}>" for l, r in intra_pairs(lhs)])}')
print(f'intra-pairs_old(lhs) -> {" ".join([f"<{l}, {r}>" for l, r in intra_pairs(rhs)])}')
print(f'intra-pairs similarity(lhs, rhs) = {intra_pairs_similarity(lhs, rhs)}')

intra-pairs_old(lhs) -> <0, 1> <2, 4> <3, 4> <2, 3> <6, 7>
intra-pairs_old(lhs) -> <0, 1> <2, 4> <3, 4> <5, 7> <2, 3> <6, 7> <5, 6>
intra-pairs similarity(lhs, rhs) = 0.8333333333333334
